# Hybrid Quantum Convolutional Neural Netwoks for Earth Observation Classification
![](https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/4609443/9656571/9647979/sebas9-3134785-large.gif)


Please refer to the following articles to get more insight about this topic:
    - Sebastianelli, A., Zaidenberg, D. A., Spiller, D., Le Saux, B., & Ullo, S. L. (2021). On circuit-based hybrid quantum neural networks for remote sensing imagery classification. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 15, 565-580.
    - Zaidenberg, D. A., Sebastianelli, A., Spiller, D., Le Saux, B., & Ullo, S. L. (2021, July). Advantages and bottlenecks of quantum machine learning for remote sensing. In 2021 IEEE International Geoscience and Remote Sensing Symposium IGARSS (pp. 5680-5683). IEEE.
    - Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.
    - Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 IEEE International Geoscience and Remote Sensing Symposium, 2018.
    - https://qiskit.org/documentation/machine-learning/tutorials/index.html
    - https://pennylane.ai/qml/demos_qml.html


# Download the dataset

In this study, we address the challenge of land use and land cover classification using Sentinel-2 satellite images. The Sentinel-2 satellite images are openly and freely accessible provided in the Earth observation program Copernicus. We present a novel dataset based on Sentinel-2 satellite images covering 13 spectral bands and consisting out of 10 classes with in total 27,000 labeled and geo-referenced images. We provide benchmarks for this novel dataset with its spectral bands using state-of-the-art deep Convolutional Neural Network (CNNs). With the proposed novel dataset, we achieved an overall classification accuracy of 98.57%. The resulting classification system opens a gate towards a number of Earth observation applications. We demonstrate how this classification system can be used for detecting land use and land cover changes and how it can assist in improving geographical maps.

Get more information [here](https://github.com/phelber/EuroSAT).

![](https://github.com/phelber/EuroSAT/blob/master/eurosat_overview_small.jpg?raw=true)




## Cloning repository with base code
Basic functions are wrapped in seprated files to keep the notebook clean. Feel free to explore the code.

In [ ]:
import sys
!rm -r /content/QML-tutorial
!git clone https://github.com/alessandrosebastianelli/QML-tutorial.git
if '/content/QML-tutorial' not in sys.path: sys.path.append('/content/QML-tutorial')


For this tutorial we are using the EuroSAT RGB dataset, accessible via wget. The following cell takes care of downloading, unzipping and preparing the dataset.

In [ ]:
!bash /content/QML-tutorial/download.sh

## Install missing packages

In [11]:
!pip install --upgrade hqm --no-deps -q

# Network train and validation

Initialize hybrid model

In [24]:
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import numpy as np
import torch

from hqm.circuits.angleencoding import StronglyEntangledCircuit, BasicEntangledCircuit
from hqm.layers.quanvolution import Quanvolution2D
import pennylane as qml

In [25]:
class HybridNet(pl.LightningModule):

    def __init__(self):
        super(HybridNet, self).__init__()
        dev = qml.device("lightning.qubit", wires=NUM_QUBITS)
        qcircuit = BasicEntangledCircuit(n_qubits=NUM_QUBITS, n_layers=NUM_LAYERS, dev=dev)
        qlayer = BasicLayer(qcircuit, aiframework='torch')
        self.network = HybridLeNet5(qlayer=qlayer, in_shape=(3,64,64), ou_dim=10)
        self.loss = torch.nn.CrossEntropyLoss()

    def forward(self, x):
        return self.network.forward(x)

        
    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)

        loss      = self.loss(outputs, labels)
        # For example, log accuracy
        _, predicted = torch.max(outputs.data, 1)
        accuracy = torch.sum(predicted == labels.data).item() / labels.size(0)

        # Logging info
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        self.log('train_accuracy', accuracy, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        
        loss      = self.loss(outputs, labels)
        # For example, log accuracy
        _, predicted = torch.max(outputs.data, 1)
        accuracy = torch.sum(predicted == labels.data).item() / labels.size(0)

        # Logging info
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log('val_accuracy', accuracy, on_epoch=True, prog_bar=True)
        
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0001)


In [27]:
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl
import torch
import sys
import os

from dataio.loader import EuroSATDataModule


torch.set_float32_matmul_precision('high')

data_module = EuroSATDataModule(num_workers=16, batch_size=8)

tb_logger = pl.loggers.TensorBoardLogger(os.path.join('lightning_logs','classifiers'), name='EuroSATClassifier')

# Instantiate ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    dirpath=os.path.join('saved_models','classifiers'),
    filename='EuroSATClassifier',
    monitor='val_loss',
    save_top_k=1,
    mode='min',
)

# Instantiate LightningModule and DataModule
model = HybridNet()

# Instantiate Trainer
trainer = pl.Trainer(max_epochs=30, callbacks=[checkpoint_callback], logger=tb_logger, accelerator="cpu")

# Train the model
trainer.fit(model, data_module)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (mps), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type             | Params
---------------------------------------------
0 | network | HybridLeNet5     | 3.0 M 
1 | loss    | CrossEntropyLoss | 0     
---------------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
11.822    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]